In [1]:
import pypsa, numpy as np
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
import linopy
#import highspy
#h=highspy.Highs()

Several Eastern Caribbean countries with  storage and links and hydrogen production as options, one year of hourly periods, with varying loads and generation from VRE 



In [2]:
#choose the working folders for reading in data and for saving output and figures
csv_folder_name =  "Eastern_CaribbeanExtendableLinksStorageHydrogen/"
network = pypsa.Network(csv_folder_name)
scenario = "LinksStorageHydrogen-base/"

INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`
INFO:pypsa.io:Imported network Eastern_CaribbeanExtendableLinksStorageHydrogen has buses, generators, links, loads, storage_units, stores


In [ ]:
#specify use of linopy and solver of choice
#network.optimize()
network.lopf(network.snapshots)


INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 28.18s
INFO:pypsa.linopf:Solve linear problem using Glpk solver


In [ ]:
#specify folder for saving results, plots
pypsa.Network.export_to_csv_folder(network,scenario)

In [ ]:
#print the load active power (P) consumption
network.loads_t.p.plot(figsize = (15,7))

In [ ]:
#print the generator active power (P) dispatch if desired and optimal capacities of generators
network.generators_t.p.plot(figsize = (15,7))
network.generators.p_nom_opt

In [ ]:
#start manipulation of results for capacities
df_cap=pd.DataFrame(network.generators.p_nom_opt)
df_cap = df_cap.reset_index()

In [ ]:
#reshape dataframe for output capacities
df_cap.columns = ['name','Capacity']
new = df_cap["name"].str.split(" ", n = 1, expand = True)
df_cap["Technology"]= new[1]
df_cap['Country']= new[0]
df_cap.drop(columns =["name"], inplace = True)
df_cap= df_cap.pivot(index = 'Country',columns = 'Technology', values = 'Capacity')
df_cap

In [ ]:
#plot capacity by country and technology
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_cap.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.tick_params(axis='both', which='major', labelsize=15)
axs.set_title("Capacity by Technology and Country",fontsize=24)
axs.set_ylabel("Capacity [MW]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel("Country",fontsize=20) 
fig.savefig(scenario+"capacity.png", bbox_inches='tight')  

In [ ]:
#scale capacities to the maximum for each country
df_cap_scaled = df_cap.div(df_cap.sum(axis=1), axis=0)*100
df_cap_scaled

In [ ]:
#plot scaled capacities by country and technology
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_cap_scaled.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.tick_params(axis='both', which='major', labelsize=15)
axs.set_title("Scaled Capacity by Technology and Country",fontsize=24)
axs.set_ylabel("Scaled Capacity [%]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel("Country",fontsize=20) 
axs.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.savefig(scenario+"scaled_capacity.png", bbox_inches='tight')

In [ ]:
#optimum storage unit capacity [MW]
network.storage_units.p_nom_opt

In [ ]:
#storage units energy capacity [MWh]
network.storage_units.p_nom_opt*network.storage_units.max_hours

In [ ]:
#print the capacities of the links, including hydrogen electrolyzers and fuel cells
network.links.p_nom_opt

In [ ]:
#print capacities of the hydrogen storages
network.stores.e_nom_opt

In [ ]:
#print capital costs of the battery storage units
network.storage_units.capital_cost

In [ ]:
#calculate total storage cost by country
network.storage_units.p_nom_opt*network.storage_units.capital_cost

In [ ]:
(network.generators.p_nom_opt*network.generators.capital_cost).sum()

In [ ]:
(network.generators_t.p.sum()*network.generators.marginal_cost).sum()

In [ ]:
#calculate total generation for the region
(network.generators_t.p.sum()).sum()

In [ ]:
#sum up generator costs by country and technology
network.generators_t.p.sum()

In [ ]:
#reshape dataframe for storage costs by country 
df_storage_cost = pd.DataFrame(network.storage_units.p_nom_opt*network.storage_units.capital_cost)
df_storage_cost.columns = ['storage_cost']
df_storage_cost = df_storage_cost.reset_index()
df_storage_cost

In [ ]:
#reshape storage costs dataframe
new = df_storage_cost["name"].str.split(" ", n = 1, expand = True)
df_storage_cost["Tech"]= new[1]
df_storage_cost['Country']= new[0]
df_storage_cost.drop(columns =["name"], inplace = True)
df_storage_cost= df_storage_cost.pivot(index = 'Country',columns = 'Tech', values = 'storage_cost')
df_storage_cost 

In [ ]:
#reshape dataframe for hydrogen tank storage costs by country 
df_hydrogen_storage_cost = pd.DataFrame(network.stores.e_nom_opt*network.stores.capital_cost)
df_hydrogen_storage_cost.columns = ['tank_cost']
df_hydrogen_storage_cost = df_hydrogen_storage_cost.reset_index()
#df_hydrogen_storage_cost
new2 = df_hydrogen_storage_cost["name"].str.split(" ", n = 1, expand = True)
df_hydrogen_storage_cost["Tech"]= new2[1]
df_hydrogen_storage_cost['Country']= new2[0]
#df_hydrogen_storage_cost
df_hydrogen_storage_cost.drop(columns =["name"], inplace = True)
df_hydrogen_storage_cost= df_hydrogen_storage_cost.pivot(index = 'Country',columns = 'Tech', values = 'tank_cost')
#df_hydrogen_storage_cost
#reshape dataframe for hydrogen electrolyzer and fuel cell costs by country 
df_hydrogen_techs_cost = pd.DataFrame(network.links.p_nom_opt*network.links.capital_cost)
df_hydrogen_techs_cost.columns = ['tank_cost']
df_hydrogen_techs_cost = df_hydrogen_techs_cost.reset_index()
#df_hydrogen_techs_cost
#reshape hydrogen electrolyzer and fuel cell costs dataframe
new3 = df_hydrogen_techs_cost["name"].str.split(" ", n = 1, expand = True)
df_hydrogen_techs_cost["Tech"]= new3[1]
df_hydrogen_techs_cost['Country']= new3[0]
#df_hydrogen_storage_cost
df_hydrogen_techs_cost.drop(columns =["name"], inplace = True)
df_hydrogen_techs_cost= df_hydrogen_techs_cost.pivot(index = 'Country',columns = 'Tech', values = 'tank_cost')
#df_hydrogen_techs_cost


In [ ]:
#reshape dataframe for generator costs
df_cost=pd.DataFrame((network.generators.p_nom_opt*network.generators.capital_cost)+(network.generators_t.p.sum()*network.generators.marginal_cost))
df_cost.columns = ['generator_cost']
df_cost = df_cost.reset_index()
#df_cost

In [ ]:
#reshape dataframe for generator and storage costs
new1 = df_cost["name"].str.split(" ", n = 1, expand = True)
df_cost["Technology"]= new1[1]
df_cost['Country']= new1[0]
df_cost.drop(columns =["name"], inplace = True)
df_cost = df_cost.pivot(index = 'Country',columns = 'Technology', values = 'generator_cost')
#extracted_col = pd.DataFrame(df_storage_cost['storage_cost'])
#extracted_col
df_cost = df_cost.join(df_storage_cost['Storage'])
df_cost = df_cost.join(df_hydrogen_storage_cost['H2 Tank'])
df_cost = df_cost.join(df_hydrogen_techs_cost['Electrolyzer'])
df_cost = df_cost.join(df_hydrogen_techs_cost['Fuel Cell'])
df_cost


In [ ]:
#reshape dataframe for actual generation by country and technology (in GWh)
df_gen=pd.DataFrame(network.generators_t.p.sum()/1000)
df_gen = df_gen.reset_index()
df_gen.columns = ['name','Electricity']
new = df_gen["name"].str.split(" ", n = 1, expand = True)
df_gen["Technology"]= new[1]
df_gen['Country']= new[0]
df_gen.drop(columns =["name"], inplace = True)
df_gen= df_gen.pivot(index = 'Country',columns = 'Technology', values = 'Electricity')
df_gen

In [ ]:
#reshape link costs dataframe
df_link_cost = pd.DataFrame(network.links.p_nom_opt*network.links.capital_cost)
df_link_cost.columns = ['link_cost']
df_link_cost = df_link_cost.reset_index()
df_link_cost

In [ ]:
#calculate total link costs
df_link_cost.link_cost.sum()

In [ ]:
#calculate LCOE for each country separately
df_cost.sum(axis=1)/df_gen.sum(axis=1)/1000

In [ ]:
#plot the actual generation by country and technology [GWh]
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_gen.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.tick_params(axis='both', which='major', labelsize=15)
axs.set_title("Generation by Technology and Country",fontsize=24)
axs.set_ylabel("Generation [GWh]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel("Country",fontsize=20) 
fig.savefig(scenario+"generation.png", bbox_inches='tight') 

In [ ]:
#scaled the generation output by country and technology
df_gen_scaled = df_gen.div(df_gen.sum(axis=1), axis=0)*100

In [ ]:
#plot the scaled output by technology and country
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_gen_scaled.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.tick_params(axis='both', which='major', labelsize=15)
axs.set_title("Scaled Generation by Technology and Country",fontsize=24)
axs.set_ylabel("Scaled Generation [%]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel("Country",fontsize=20) 
axs.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.savefig(scenario+"scaled_generation.png", bbox_inches='tight') 

In [ ]:
#network.generators_t.p.sum()/(network.generators_t.p.sum()).sum()

In [ ]:
#calculate capacity factors by country and technology
network.generators_t.p.sum()/(network.generators.p_nom_opt*8760)

In [ ]:
#calculate regional LCOE
((network.generators.p_nom_opt*network.generators.capital_cost).sum()+(network.generators_t.p.sum()*network.generators.marginal_cost).sum()+(network.storage_units.p_nom_opt*network.storage_units.capital_cost).sum()+(network.stores.e_nom_opt*network.stores.capital_cost).sum()+(network.links.p_nom_opt*network.links.capital_cost).sum()).sum()/(network.loads_t.p_set.sum()).sum()

In [ ]:
#print storage units
network.storage_units_t.p.plot(figsize = (15,7))

